In [ ]:
# Import necessary libraries
import numpy as np
from itertools import product
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dataset_sabr import generate_sabr_call_options

# Problem Surface Dataset

In [ ]:
# Maturity times and strike prices from the previous product grid setup
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
# maturity_time_list = np.logspace(np.log10(0.02), np.log10(5), 100)
# strike_price_list = np.logspace(np.log10(0.5), np.log10(2), 100)

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Display the resulting dataset
call_option_dataset

In [ ]:
# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)

# Generate the dataset using the SABR model and Black-Scholes formula
hypothetical_call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=hypothetical_maturity_times,
    strike_prices=hypothetical_strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Problem dataset
maturity_times_scatter = call_option_dataset["Time to Maturity"].values
strike_prices_scatter = call_option_dataset["Strike Price"].values
call_prices_scatter = call_option_dataset["Call Option Price"].values
implied_volatilities_scatter = call_option_dataset["Implied Volatility"].values

# Hypothetical dataset
hypothetical_maturity_times = hypothetical_call_option_dataset["Time to Maturity"].values
hypothetical_strike_prices = hypothetical_call_option_dataset["Strike Price"].values
hypothetical_call_prices = hypothetical_call_option_dataset["Call Option Price"].values
hypothetical_implied_volatilities = hypothetical_call_option_dataset["Implied Volatility"].values

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))
hypothetical_call_prices_grid = hypothetical_call_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))
hypothetical_implied_volatilities_grid = hypothetical_implied_volatilities.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

# Create the 1x2 subplot figure
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]],  # Surface plots in both subplots
    subplot_titles=("Call Option Price Surface", "Implied Volatility Surface"),
)

# Plot 3D surface for call option prices
call_option_surface = go.Surface(
    x=hypothetical_strikes_grid,
    y=hypothetical_maturities_grid,
    z=hypothetical_call_prices_grid,
    # colorscale='Viridis',
    name='Call Option Price',
    colorbar=dict(title='Price', x=0.45),
)

# Plot 3D surface for implied volatility
implied_vol_surface = go.Surface(
    x=hypothetical_strikes_grid,
    y=hypothetical_maturities_grid,
    z=hypothetical_implied_volatilities_grid,
    colorscale='Viridis',
    name='Implied Volatility',
    colorbar=dict(title='IV', x=1),
)

# Add scatter points for call option prices from the real dataset
call_option_scatter = go.Scatter3d(
    x=strike_prices_scatter,
    y=maturity_times_scatter,
    z=call_prices_scatter,
    mode='markers',
    marker=dict(size=4, color='black', symbol='circle'),
)

# Add scatter points for implied volatilities from the real dataset
implied_vol_scatter = go.Scatter3d(
    x=strike_prices_scatter,
    y=maturity_times_scatter,
    z=implied_volatilities_scatter,
    mode='markers',
    marker=dict(size=4, color='black', symbol='circle'),
)

# Add the surfaces to the figure
fig.add_trace(call_option_surface, row=1, col=1)
fig.add_trace(implied_vol_surface, row=1, col=2)

# Add scatter points on top of the surfaces
fig.add_trace(call_option_scatter, row=1, col=1)
fig.add_trace(implied_vol_scatter, row=1, col=2)

# Update layout for better visualization
fig.update_layout(
    title="SABR Call Option Prices and Implied Volatility",
    height=750,
    width=1500,
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Call Option Price',
        aspectratio=dict(x=1, y=1, z=0.7)
    ),
    scene2=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Implied Volatility',
        aspectratio=dict(x=1, y=1, z=0.7)
    ),
    showlegend=False
)

# Show the figure
fig.show()

In [ ]:
fig.write_image('figs/sabr_dataset.png', format='png', scale=4, width=1500, height=750)